# सत्र ५ – बहु-एजेन्ट समन्वयकर्ता

Foundry Local प्रयोग गरेर एक साधारण दुई-एजेन्ट पाइपलाइन (अनुसन्धानकर्ता -> सम्पादक) प्रदर्शन गर्दछ।


### व्याख्या: निर्भरता स्थापना
स्थानीय मोडेल पहुँच र च्याट पूरा गर्न आवश्यक `foundry-local-sdk` र `openai` स्थापना गर्दछ। पुनरावृत्त योग्य।


# परिदृश्य
दुई-एजेन्टको न्यूनतम समन्वय ढाँचा कार्यान्वयन:
- **अनुसन्धानकर्ता एजेन्ट**ले संक्षिप्त तथ्यगत बुलेटहरू सङ्कलन गर्छ
- **सम्पादक एजेन्ट**ले कार्यकारी स्पष्टताका लागि पुनःलेखन गर्छ

एजेन्टहरूबीच साझा स्मृति, मध्यवर्ती परिणामको क्रमिक पासिङ, र साधारण पाइपलाइन कार्यलाई प्रदर्शन गर्दछ। थप भूमिकाहरू (जस्तै, आलोचक, प्रमाणिकर्ता) वा समानान्तर शाखाहरूमा विस्तार गर्न सकिन्छ।

**पर्यावरण चरहरू:**
- `FOUNDRY_LOCAL_ALIAS` - प्रयोग गर्नुपर्ने डिफल्ट मोडेल (डिफल्ट: phi-4-mini)
- `AGENT_MODEL_PRIMARY` - प्राथमिक एजेन्ट मोडेल (ALIAS लाई ओभरराइड गर्छ)
- `AGENT_MODEL_EDITOR` - सम्पादक एजेन्ट मोडेल (डिफल्टमा प्राथमिक मोडेल प्रयोग गर्छ)

**SDK सन्दर्भ:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local

**यसले कसरी काम गर्छ:**
1. **FoundryLocalManager** ले स्वचालित रूपमा Foundry Local सेवा सुरु गर्छ
2. निर्दिष्ट मोडेल डाउनलोड र लोड गर्छ (वा क्यास गरिएको संस्करण प्रयोग गर्छ)
3. अन्तरक्रियाका लागि OpenAI-संगत अन्तर्क्रियात्मक अन्तबिन्दु प्रदान गर्छ
4. प्रत्येक एजेन्टले विशेष कार्यहरूको लागि फरक मोडेल प्रयोग गर्न सक्छ
5. बिल्ट-इन पुनः प्रयास गर्ने तर्कले अस्थायी असफलताहरूलाई सहज रूपमा व्यवस्थापन गर्छ

**मुख्य विशेषताहरू:**
- ✅ स्वचालित सेवा पत्ता लगाउने र आरम्भ गर्ने क्षमता
- ✅ मोडेल जीवनचक्र व्यवस्थापन (डाउनलोड, क्यास, लोड)
- ✅ परिचित API का लागि OpenAI SDK अनुकूलता
- ✅ एजेन्ट विशेषज्ञताका लागि बहु-मोडेल समर्थन
- ✅ पुनः प्रयास गर्ने तर्कसहितको बलियो त्रुटि व्यवस्थापन
- ✅ स्थानीय अनुमान (कुनै क्लाउड API आवश्यक छैन)


In [16]:
# Install dependencies
!pip install -q foundry-local-sdk openai

### व्याख्या: कोर आयातहरू र टाइपिङ
एजेन्ट सन्देश भण्डारणका लागि dataclasses र स्पष्टताको लागि टाइपिङ संकेतहरू प्रस्तुत गर्दछ। एजेन्टको आगामी कार्यहरूको लागि Foundry Local प्रबन्धक र OpenAI क्लाइन्ट आयात गर्दछ।


In [17]:
from dataclasses import dataclass, field
from typing import List
import os
from foundry_local import FoundryLocalManager
from openai import OpenAI

### व्याख्या: मोडेल आरम्भ (SDK ढाँचा)
Foundry Local Python SDK लाई बलियो मोडेल व्यवस्थापनका लागि प्रयोग गरिन्छ:
- **FoundryLocalManager(alias)** - सेवा स्वचालित रूपमा सुरु गर्छ र alias द्वारा मोडेल लोड गर्छ
- **get_model_info(alias)** - alias लाई ठोस मोडेल ID मा रूपान्तरण गर्छ
- **manager.endpoint** - OpenAI क्लाइन्टका लागि सेवा endpoint प्रदान गर्छ
- **manager.api_key** - API key प्रदान गर्छ (स्थानीय प्रयोगका लागि वैकल्पिक)
- फरक एजेन्टहरूका लागि छुट्टाछुट्टै मोडेलहरूको समर्थन (प्राथमिक बनाम सम्पादक)
- लचिलोपनका लागि एक्स्पोनेन्सियल ब्याकअफसहितको बिल्ट-इन पुन: प्रयास तर्क
- सेवा तयार छ भनी सुनिश्चित गर्न जडान प्रमाणीकरण

**मुख्य SDK ढाँचा:**
```python
manager = FoundryLocalManager(alias)
model_info = manager.get_model_info(alias)
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key)
```

**जीवनचक्र व्यवस्थापन:**
- व्यवस्थापकहरूलाई उचित सफाइका लागि विश्वव्यापी रूपमा भण्डारण गरिन्छ
- प्रत्येक एजेन्टले विशेषज्ञताका लागि फरक मोडेल प्रयोग गर्न सक्छ
- स्वचालित सेवा पत्ता लगाउने र जडान व्यवस्थापन
- असफलतामा एक्स्पोनेन्सियल ब्याकअफसहितको सहज पुन: प्रयास

यसले एजेन्ट समन्वय सुरु हुनु अघि उचित आरम्भ सुनिश्चित गर्दछ।

**सन्दर्भ:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local


In [18]:
import time

# Environment configuration
PRIMARY_ALIAS = os.getenv('AGENT_MODEL_PRIMARY', os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini'))
EDITOR_ALIAS = os.getenv('AGENT_MODEL_EDITOR', PRIMARY_ALIAS)

# Store managers globally for proper lifecycle management
primary_manager = None
editor_manager = None

def init_model(alias: str, max_retries: int = 3):
    """Initialize Foundry Local manager with retry logic.
    
    Args:
        alias: Model alias to initialize
        max_retries: Number of retry attempts with exponential backoff
    
    Returns:
        Tuple of (manager, client, model_id, endpoint)
    """
    delay = 2.0
    last_err = None
    
    for attempt in range(1, max_retries + 1):
        try:
            print(f"[Init] Starting Foundry Local for '{alias}' (attempt {attempt}/{max_retries})...")
            
            # Initialize manager - this starts the service and loads the model
            manager = FoundryLocalManager(alias)
            
            # Get model info to retrieve the actual model ID
            model_info = manager.get_model_info(alias)
            model_id = model_info.id
            
            # Create OpenAI client with manager's endpoint
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key or 'not-needed'
            )
            
            # Verify the connection with a simple test
            models = client.models.list()
            print(f"[OK] Initialized '{alias}' -> {model_id} at {manager.endpoint}")
            
            return manager, client, model_id, manager.endpoint
            
        except Exception as e:
            last_err = e
            if attempt < max_retries:
                print(f"[Retry {attempt}/{max_retries}] Failed to init '{alias}': {e}")
                print(f"[Retry] Waiting {delay:.1f}s before retry...")
                time.sleep(delay)
                delay *= 2
            else:
                print(f"[ERROR] Failed to initialize '{alias}' after {max_retries} attempts")
    
    raise RuntimeError(f"Failed to initialize '{alias}' after {max_retries} attempts: {last_err}")

# Initialize primary model (for researcher)
print(f"\n{'='*80}")
print(f"Initializing Primary Model: {PRIMARY_ALIAS}")
print('='*80)
primary_manager, primary_client, PRIMARY_MODEL_ID, primary_endpoint = init_model(PRIMARY_ALIAS)

# Initialize editor model (may be same as primary)
if EDITOR_ALIAS != PRIMARY_ALIAS:
    print(f"\n{'='*80}")
    print(f"Initializing Editor Model: {EDITOR_ALIAS}")
    print('='*80)
    editor_manager, editor_client, EDITOR_MODEL_ID, editor_endpoint = init_model(EDITOR_ALIAS)
else:
    print(f"\n[Info] Editor using same model as primary")
    editor_manager = primary_manager
    editor_client, EDITOR_MODEL_ID = primary_client, PRIMARY_MODEL_ID
    editor_endpoint = primary_endpoint

print(f"\n{'='*80}")
print(f"[Configuration Summary]")
print('='*80)
print(f"  Primary Agent:")
print(f"    - Alias: {PRIMARY_ALIAS}")
print(f"    - Model: {PRIMARY_MODEL_ID}")
print(f"    - Endpoint: {primary_endpoint}")
print(f"\n  Editor Agent:")
print(f"    - Alias: {EDITOR_ALIAS}")
print(f"    - Model: {EDITOR_MODEL_ID}")
print(f"    - Endpoint: {editor_endpoint}")
print('='*80)



Initializing Primary Model: phi-4-mini
[Init] Starting Foundry Local for 'phi-4-mini' (attempt 1/3)...
[OK] Initialized 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4 at http://127.0.0.1:59959/v1

Initializing Editor Model: gpt-oss-20b
[Init] Starting Foundry Local for 'gpt-oss-20b' (attempt 1/3)...
[OK] Initialized 'gpt-oss-20b' -> gpt-oss-20b-cuda-gpu:1 at http://127.0.0.1:59959/v1

[Configuration Summary]
  Primary Agent:
    - Alias: phi-4-mini
    - Model: Phi-4-mini-instruct-cuda-gpu:4
    - Endpoint: http://127.0.0.1:59959/v1

  Editor Agent:
    - Alias: gpt-oss-20b
    - Model: gpt-oss-20b-cuda-gpu:1
    - Endpoint: http://127.0.0.1:59959/v1


### व्याख्या: एजेन्ट र मेमोरी कक्षाहरू
हल्का `AgentMsg` मेमोरी प्रविष्टिहरूको लागि परिभाषित गर्दछ र `Agent` लाई समेट्छ:
- **सिस्टम भूमिका** - एजेन्टको व्यक्तित्व र निर्देशनहरू
- **सन्देश इतिहास** - संवादको सन्दर्भलाई कायम राख्छ
- **act() विधि** - उचित त्रुटि व्यवस्थापनसहित कार्यहरू कार्यान्वयन गर्दछ

एजेन्टले विभिन्न मोडेलहरू (प्राथमिक बनाम सम्पादक) प्रयोग गर्न सक्छ र प्रत्येक एजेन्टको लागि अलग सन्दर्भ कायम राख्छ। यस ढाँचाले निम्नलाई सक्षम बनाउँछ:
- कार्यहरू बीच मेमोरीको निरन्तरता
- प्रत्येक एजेन्टको लागि लचिलो मोडेल असाइनमेन्ट
- त्रुटि अलगाव र पुन: प्राप्ति
- सजिलो चेनिङ र समन्वय


In [19]:
@dataclass
class AgentMsg:
    role: str
    content: str

@dataclass
class Agent:
    name: str
    system: str
    client: OpenAI = None  # Allow per-agent client assignment
    model_id: str = None   # Allow per-agent model
    memory: List[AgentMsg] = field(default_factory=list)

    def _history(self):
        """Return chat history in OpenAI messages format including system + memory."""
        msgs = [{'role': 'system', 'content': self.system}]
        for m in self.memory[-6:]:  # Keep last 6 messages to avoid context overflow
            msgs.append({'role': m.role, 'content': m.content})
        return msgs

    def act(self, prompt: str, temperature: float = 0.4, max_tokens: int = 300):
        """Send a prompt, store user + assistant messages in memory, and return assistant text.
        
        Args:
            prompt: User input/task for the agent
            temperature: Sampling temperature (0.0-1.0)
            max_tokens: Maximum tokens to generate
        
        Returns:
            Assistant response text
        """
        # Use agent-specific client/model or fall back to primary
        client_to_use = self.client or primary_client
        model_to_use = self.model_id or PRIMARY_MODEL_ID
        
        self.memory.append(AgentMsg('user', prompt))
        
        try:
            # Build messages including system prompt and history
            messages = self._history() + [{'role': 'user', 'content': prompt}]
            
            resp = client_to_use.chat.completions.create(
                model=model_to_use,
                messages=messages,
                max_tokens=max_tokens,
                temperature=temperature,
            )
            
            # Validate response
            if not resp.choices:
                raise RuntimeError("No completion choices returned")
            
            out = resp.choices[0].message.content or ""
            
            if not out:
                raise RuntimeError("Empty response content")
            
        except Exception as e:
            out = f"[ERROR:{self.name}] {type(e).__name__}: {str(e)}"
            print(f"[Agent Error] {self.name}: {type(e).__name__}: {str(e)}")
        
        self.memory.append(AgentMsg('assistant', out))
        return out

print("[INFO] Agent classes initialized with Foundry SDK support")
print(f"[INFO] Using OpenAI SDK version: {OpenAI.__module__}")


[INFO] Agent classes initialized with Foundry SDK support
[INFO] Using OpenAI SDK version: openai


### व्याख्या: समन्वित पाइपलाइन
दुई विशेष एजेन्टहरू सिर्जना गर्दछ:
- **अनुसन्धाता**: प्राथमिक मोडेल प्रयोग गर्दछ, तथ्यगत जानकारी संकलन गर्दछ
- **सम्पादक**: छुट्टै मोडेल प्रयोग गर्न सक्छ (यदि कन्फिगर गरिएको छ भने), परिष्कृत र पुनर्लेखन गर्दछ

`pipeline` फङ्सन:
1. अनुसन्धाताले कच्चा जानकारी संकलन गर्दछ
2. सम्पादकले कार्यकारी-तयार परिणाममा परिष्कृत गर्दछ
3. दुबै मध्यवर्ती र अन्तिम परिणामहरू फर्काउँछ

यो ढाँचाले सक्षम बनाउँछ:
- मोडेल विशेषज्ञता (विभिन्न भूमिकाहरूका लागि विभिन्न मोडेलहरू)
- बहु-चरणीय प्रशोधन मार्फत गुणस्तर सुधार
- जानकारी रूपान्तरणको ट्रेसबिलिटी
- थप एजेन्टहरू वा समानान्तर प्रशोधनमा सजिलो विस्तार


In [ ]:
# Create specialized agents with optional model assignment
researcher = Agent(
    name='Researcher',
    system='You collect concise factual bullet points.',
    client=primary_client,
    model_id=PRIMARY_MODEL_ID
)

editor = Agent(
    name='Editor',
    system='You rewrite content for clarity and an executive, action-focused tone.',
    client=editor_client,
    model_id=EDITOR_MODEL_ID
)

def pipeline(q: str, verbose: bool = True):
    """Execute multi-agent pipeline: Researcher -> Editor.
    
    Args:
        q: User question/task
        verbose: Print intermediate outputs
    
    Returns:
        Dictionary with research, final outputs, and metadata
    """
    if verbose:
        print(f"[Pipeline] Question: {q}\n")
    
    # Stage 1: Research
    if verbose:
        print("[Stage 1: Research]")
    research = researcher.act(q)
    if verbose:
        print(f"Output: {research[:200]}...\n")
    
    # Stage 2: Editorial refinement
    if verbose:
        print("[Stage 2: Editorial Refinement]")
    rewrite = editor.act(
        f"Rewrite professionally with a 1-sentence executive summary first. "
        f"Improve clarity, keep bullet structure if present. Source:\n{research}"
    )
    if verbose:
        print(f"Output: {rewrite[:200]}...\n")
    
    return {
        'question': q,
        'research': research,
        'final': rewrite,
        'models': {
            'researcher': PRIMARY_MODEL_ID,
            'editor': EDITOR_MODEL_ID
        }
    }

# Execute sample pipeline
print("="*80)
result = pipeline('Explain why edge AI matters for compliance and latency.')
print("="*80)
print("\n[FINAL OUTPUT]")
print(result['final'])
print("\n[METADATA]")
print(f"Models used: {result['models']}")
result

[Pipeline] Question: Explain why edge AI matters for compliance and latency.

[Stage 1: Research]
Output: - **Data Sovereignty**: Edge AI allows data to be processed locally, which can help organizations comply with regional data protection regulations by keeping sensitive information within the borders o...

[Stage 2: Editorial Refinement]


### व्याख्या: पाइपलाइन कार्यान्वयन र परिणामहरू
अनुपालन + विलम्बता सम्बन्धित प्रश्नमा बहु-एजेन्ट पाइपलाइन कार्यान्वयन गरेर प्रदर्शन गरिन्छ:
- बहु-चरण जानकारी रूपान्तरण
- एजेन्टको विशेषज्ञता र सहकार्य
- परिमार्जन मार्फत उत्पादनको गुणस्तर सुधार
- ट्रेसबिलिटी (दुवै मध्यवर्ती र अन्तिम परिणामहरू सुरक्षित)

**परिणाम संरचना:**
- `question` - प्रयोगकर्ताको मूल प्रश्न
- `research` - कच्चा अनुसन्धान परिणाम (तथ्यगत बुलेटहरू)
- `final` - परिमार्जित कार्यकारी सारांश
- `models` - प्रत्येक चरणमा प्रयोग गरिएका मोडेलहरू

**विस्तारका विचारहरू:**
1. गुणस्तर समीक्षा गर्न Critic एजेन्ट थप्नुहोस्
2. विभिन्न पक्षहरूको लागि समानान्तर अनुसन्धान एजेन्टहरू कार्यान्वयन गर्नुहोस्
3. तथ्य-जाँचको लागि Verifier एजेन्ट थप्नुहोस्
4. विभिन्न जटिलता स्तरहरूको लागि विभिन्न मोडेलहरू प्रयोग गर्नुहोस्
5. पुनरावृत्त सुधारको लागि प्रतिक्रिया लूपहरू कार्यान्वयन गर्नुहोस्


### उन्नत: कस्टम एजेन्ट कन्फिगरेसन

एजेन्टको व्यवहारलाई अनुकूलन गर्न प्रयास गर्नुहोस् वातावरण चरहरू परिवर्तन गरेर सुरुवात सेल चलाउनु अघि:

**उपलब्ध मोडेलहरू:**
- टर्मिनलमा `foundry model ls` प्रयोग गरेर सबै उपलब्ध मोडेलहरू हेर्नुहोस्
- उदाहरणहरू: phi-4-mini, phi-3.5-mini, qwen2.5-7b, llama-3.2-3b, आदि।


In [ ]:
# Example: Use different models for different agents
# Uncomment and modify as needed:

# import os
# os.environ['AGENT_MODEL_PRIMARY'] = 'phi-4-mini'      # Fast, good for research
# os.environ['AGENT_MODEL_EDITOR'] = 'qwen2.5-7b'       # Higher quality for editing

# Then restart the kernel and re-run all cells

# Test with different questions
test_questions = [
    "What are 3 key benefits of using small language models?",
    "How does RAG improve AI accuracy?",
    "Why is local inference important for privacy?"
]

print("Testing pipeline with multiple questions:\n")
for i, q in enumerate(test_questions, 1):
    print(f"\n{'='*80}")
    print(f"Question {i}: {q}")
    print('='*80)
    r = pipeline(q, verbose=False)
    print(f"\n[FINAL]: {r['final'][:300]}...")
    print(f"[Models]: Researcher={r['models']['researcher']}, Editor={r['models']['editor']}")


Testing pipeline with multiple questions:


Question 1: What are 3 key benefits of using small language models?

[FINAL]: <|channel|>analysis<|message|>The user wants a rewrite of the entire block of text. The rewrite should be professional, include a one-sentence executive summary first, improve clarity, keep bullet structure if present. The user has provided a large amount of text. The user wants a rewrite of that te...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 2: How does RAG improve AI accuracy?

[FINAL]: <|channel|>final<|message|>**RAG (Retrieval‑Augmented Generation) empowers AI to produce highly accurate, contextually relevant responses by combining a retrieval system with a large language model (LLM).**<|return|>...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 3: Why is local inference important for privacy?

[FINAL]: <|channel|>final<|message|>**Local inference—processing data d


---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरी अनुवाद गरिएको हो। हामी यथासम्भव सटीकता सुनिश्चित गर्न प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादहरूमा त्रुटिहरू वा अशुद्धताहरू हुन सक्छन्। यसको मूल भाषामा रहेको मूल दस्तावेज़लाई आधिकारिक स्रोत मानिनुपर्छ। महत्वपूर्ण जानकारीका लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याको लागि हामी जिम्मेवार हुने छैनौं।
